# Performance test without native execution engine using the NYC taxi - yellow dataset from https://learn.microsoft.com/en-us/azure/open-datasets/dataset-taxi-yellow?tabs=pyspark
In this example, I am using a Fabric environment with the native execution engine disabled.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

StatementMeta(, 57d190c8-f1fc-49dd-b0c8-6432484e6f1b, 3, Finished, Available, Finished)

In [2]:
# Create Spark session
spark = SparkSession.builder \
    .appName("NativeExecutionEngine") \
    .getOrCreate()

StatementMeta(, 57d190c8-f1fc-49dd-b0c8-6432484e6f1b, 4, Finished, Available, Finished)

In [3]:
# Get the NYC Taxi yellow data from Microsoft: https://learn.microsoft.com/en-us/azure/open-datasets/dataset-taxi-yellow?tabs=pyspark

# Azure storage access info
blob_account_name = "azureopendatastorage"
blob_container_name = "nyctlc"
blob_relative_path = "yellow"
blob_sas_token = r""

# Allow SPARK to read from Blob remotely
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
spark.conf.set(
  'fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name),
  blob_sas_token)
print('Remote blob path: ' + wasbs_path)

# SPARK read parquet, note that it won't load any data yet by now
df_yellow = spark.read.parquet(wasbs_path)
print('Register the DataFrame as a SQL temporary view: source')
df_yellow.createOrReplaceTempView('yellow')

StatementMeta(, 57d190c8-f1fc-49dd-b0c8-6432484e6f1b, 5, Finished, Available, Finished)

Remote blob path: wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/yellow
Register the DataFrame as a SQL temporary view: source


In [5]:
# Count the number of rows
df_yellow.count()

StatementMeta(, 57d190c8-f1fc-49dd-b0c8-6432484e6f1b, 7, Finished, Available, Finished)

1571671152

In [7]:
# Executes a SQL query using Spark SQL to sum up all values in the 'totalAmount' column from the 'yellow' table
yellow_sum = spark.sql('SELECT SUM(totalAmount) FROM yellow')

# Triggers the execution of the query and retrieves the result as a list of Row objects
yellow_sum.collect()

# Displays the execution plan of the query
yellow_sum.explain()

StatementMeta(, 57d190c8-f1fc-49dd-b0c8-6432484e6f1b, 9, Finished, Available, Finished)

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(2) HashAggregate(keys=[], functions=[sum(totalAmount#714)])
   +- ShuffleQueryStage 0
      +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=362]
         +- *(1) HashAggregate(keys=[], functions=[partial_sum(totalAmount#714)])
            +- *(1) Project [totalAmount#714]
               +- *(1) ColumnarToRow
                  +- FileScan parquet [totalAmount#714,puYear#715,puMonth#716] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/yellow], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<totalAmount:double>
+- == Initial Plan ==
   HashAggregate(keys=[], functions=[sum(totalAmount#714)])
   +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=335]
      +- HashAggregate(keys=[], functions=[partial_sum(totalAmount#714)])
         +- Project [totalAmount#714]
            +- FileScan pa